In [ ]:
################################################################################
#                                                                              #
#       GLOBAL DISASTER ANALYTICS & IMPACT ASSESSMENT SYSTEM                   #
#                                                                              #
################################################################################
#                                                                              #
# Author: Demian Pyrih                                                         #
# Date: October 2025                                                           #
# Description: Comprehensive analysis of natural disasters worldwide           #
#              (1900-2024) with ML predictions and geospatial analytics        #
#                                                                              #
################################################################################

# ==============================================================================
# SECTION 1: IMPORT LIBRARIES
# ==============================================================================

# Data Manipulation
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, mean_squared_error, r2_score
import xgboost as xgb

# Statistical Analysis
from scipy import stats

# ==============================================================================
# SECTION 2: CONFIGURATION
# ==============================================================================

# Visualization settings
plt.style.use('default')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (14, 6)

# Pandas settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# ==============================================================================
# SECTION 3: INITIALIZATION
# ==============================================================================

print("=" * 80)
print(" " * 15 + "GLOBAL DISASTER ANALYTICS & IMPACT ASSESSMENT")
print("=" * 80)
print()
print("✅ All libraries imported successfully!")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Python Version: 3.11")
print(f"📊 Dataset: EM-DAT International Disaster Database (1900-2024)")
print()
print("=" * 80)